In [1]:
from urllib.request  import urlopen
import bs4
import datetime as dt
import pandas as pd
import re
from pandas import Series, DataFrame

In [2]:
index_cd='006400'
page_n=1
naver_index='https://finance.naver.com/item/sise_day.nhn?code=' +index_cd+'&page='+str(page_n)
from urllib.request import urlopen
source=urlopen(naver_index).read()
import bs4
source=bs4.BeautifulSoup(source,'lxml')
print(source.prettify())

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 금융
  </title>
  <link href="/css/newstock.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/common.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/layout.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/main.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock2.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock3.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/world.css?20190411143845" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <script language="JavaScript">
   function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
}
  </script>
  <h4 class="tlline2">
   <strong>
    <span class="red03">
     일별
    </span>
    시세
   </strong>
  </h

In [3]:
d=source.find_all('span',class_='tah p10 gray03')[0].text
d

'2019.05.03'

In [4]:
import datetime as dt

In [5]:
def date_format(d):
    d = str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy,mm,dd)
    return this_date

In [6]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n = 1, last_page = 0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
        
    naver_index = 'https://finance.naver.com/item/sise_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    source = bs4.BeautifulSoup(urlopen(naver_index).read(), 'lxml')
    dates=source.find_all('span',class_='tah p10 gray03')
    prices=source.find_all('td',class_='num')
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                this_close = prices[n*6].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
                historical_prices[this_date] = this_close
            elif this_date < start_date:
                return historical_prices
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        last_page = int(last_page.split('&')[1].split('=')[1])

    if page_n < last_page:
        page_n += 1
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices


In [7]:
index_cd='006400'
historical_prices=dict()
historical_index_naver(index_cd,'2017.03.01','2019.2.28')
historical_prices

{datetime.date(2019, 2, 28): 237500.0,
 datetime.date(2019, 2, 27): 239000.0,
 datetime.date(2019, 2, 26): 240500.0,
 datetime.date(2019, 2, 25): 245500.0,
 datetime.date(2019, 2, 22): 241000.0,
 datetime.date(2019, 2, 21): 243500.0,
 datetime.date(2019, 2, 20): 248000.0,
 datetime.date(2019, 2, 19): 247500.0,
 datetime.date(2019, 2, 18): 245000.0,
 datetime.date(2019, 2, 15): 236000.0,
 datetime.date(2019, 2, 14): 240500.0,
 datetime.date(2019, 2, 13): 238500.0,
 datetime.date(2019, 2, 12): 236500.0,
 datetime.date(2019, 2, 11): 236500.0,
 datetime.date(2019, 2, 8): 238500.0,
 datetime.date(2019, 2, 7): 240000.0,
 datetime.date(2019, 2, 1): 227000.0,
 datetime.date(2019, 1, 31): 223500.0,
 datetime.date(2019, 1, 30): 228000.0,
 datetime.date(2019, 1, 29): 229000.0,
 datetime.date(2019, 1, 28): 232000.0,
 datetime.date(2019, 1, 25): 232500.0,
 datetime.date(2019, 1, 24): 225500.0,
 datetime.date(2019, 1, 23): 230500.0,
 datetime.date(2019, 1, 22): 229000.0,
 datetime.date(2019, 1, 21):

In [8]:
prices={'sdi':historical_prices}
df=pd.DataFrame(prices)
df

,sdi
2017-03-02,129000.0
2017-03-03,125000.0
2017-03-06,125000.0
2017-03-07,125000.0
2017-03-08,130000.0
2017-03-09,127500.0
2017-03-10,126500.0
2017-03-13,131000.0
2017-03-14,133500.0
2017-03-15,135500.0


In [9]:
df['5_mean']=df['sdi'].rolling(window=5).mean()
df['20_mean']=df['sdi'].rolling(window=20).mean()
df

,sdi,5_mean,20_mean
2017-03-02,129000.0,NaN,NaN
2017-03-03,125000.0,NaN,NaN
2017-03-06,125000.0,NaN,NaN
2017-03-07,125000.0,NaN,NaN
2017-03-08,130000.0,126800.0,NaN
2017-03-09,127500.0,126500.0,NaN
2017-03-10,126500.0,126800.0,NaN
2017-03-13,131000.0,128000.0,NaN
2017-03-14,133500.0,129700.0,NaN
2017-03-15,135500.0,130800.0,NaN


In [10]:
전략=0
보유잔고=10000000
주식개수=0
중복=False

df['전략']=Series([전략],index=[df.index[0]])
df['주식개수']=Series([주식개수],index=[df.index[0]])
df['보유잔고']=Series([보유잔고],index=[df.index[0]])

for i in range(0, len(df)):
    if df['5_mean'][i]==None or df['20_mean'][i]==None:
        전략=0
        df['전략'][i]=전략
        df['주식개수'][i]=주식개수
        df['보유잔고'][i]=보유잔고
    # 골든크로스
    elif df['5_mean'][i]>df['20_mean'][i]:
        #전략 0이었던 거를 골든크로스가 발생했을 때 1(매수)로 바꿔주고, 첫째날은 넘어간다.
        if 전략!=1:
            전략=1
            중복=False
            df['전략'][i]=전략
            df['주식개수'][i]=주식개수
            df['보유잔고'][i]=보유잔고
            continue
        # 골든크로스 두쨋날 매수하고 그 다음일은 전부 중복,True로 바꾼다.
        elif 전략==1:        
            if 중복==False:
                주식개수=보유잔고//df['sdi'][i]
                보유잔고=보유잔고-(주식개수*df['sdi'][i])
                df['전략'][i]=전략
                df['주식개수'][i]=주식개수
                df['보유잔고'][i]=보유잔고
                중복=True
            #두번째날 다음부터는 전략,주식개수,보유잔고 유지
            elif 중복==True:
                df['전략'][i]=전략
                df['주식개수'][i]=주식개수
                df['보유잔고'][i]=보유잔고
    # 데드크로스
    elif df['5_mean'][i]<df['20_mean'][i]:
        # 데드크로스가 발생했을 때 전략 1인거를 2로 바꿔주고 다음날로 넘어감.
        if 전략!=2:
            전략=2
            중복=False
            df['전략'][i]=전략
            df['주식개수'][i]=주식개수
            df['보유잔고'][i]=보유잔고
            continue
        # 데드크로스 둘쨋날 일 때 매도하고 중복을 True로 바꿔줌
        elif 전략==2:
            if 중복==False:
                보유잔고=보유잔고+(주식개수*df['sdi'][i])
                주식개수=0
                df['전략'][i]=전략
                df['주식개수'][i]=주식개수
                df['보유잔고'][i]=보유잔고
                중복=True
            #둘쨋날 다음부터는 유지    
            elif 중복==True:
                df['전략'][i]=전략
                df['주식개수'][i]=주식개수
                df['보유잔고'][i]=보유잔고


In [11]:
df

,sdi,5_mean,20_mean,전략,주식개수,보유잔고
2017-03-02,129000.0,NaN,NaN,0.0,0.0,10000000.0
2017-03-03,125000.0,NaN,NaN,NaN,NaN,NaN
2017-03-06,125000.0,NaN,NaN,NaN,NaN,NaN
2017-03-07,125000.0,NaN,NaN,NaN,NaN,NaN
2017-03-08,130000.0,126800.0,NaN,NaN,NaN,NaN
2017-03-09,127500.0,126500.0,NaN,NaN,NaN,NaN
2017-03-10,126500.0,126800.0,NaN,NaN,NaN,NaN
2017-03-13,131000.0,128000.0,NaN,NaN,NaN,NaN
2017-03-14,133500.0,129700.0,NaN,NaN,NaN,NaN
2017-03-15,135500.0,130800.0,NaN,NaN,NaN,NaN


In [12]:
df.to_excel("빅금과제2.xlsx")